In [2]:
# automatically reload edited source code
%load_ext autoreload
%autoreload 2

# external package imports
import numpy as np
import pandas as pd
import os, pdb, glob, time, argparse
from os.path import exists, split, isdir, getsize
from sdo_clv_pipeline.sdo_plot import *

# internal package imports
from sdo_clv_pipeline.paths import root
from sdo_clv_pipeline.sdo_io import *
from sdo_clv_pipeline.sdo_process import *
from sdo_clv_pipeline.sdo_io import * 

c:\Users\srugi\Documents\sdo-clv-pipeline\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# returns four sorted lists with the path to each of the file types
data_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "data"))
con_files, mag_files, dop_files, aia_files = find_data(data_dir+"\\", "2014")
print(data_dir)
print(con_files)
print(mag_files)
print(dop_files)
print(aia_files)

c:\Users\srugi\Documents\sdo-clv-pipeline\data
['c:\\Users\\srugi\\Documents\\sdo-clv-pipeline\\data\\hmi.ic_720s.20140101_000000_TAI.1.continuum.fits', 'c:\\Users\\srugi\\Documents\\sdo-clv-pipeline\\data\\hmi.ic_720s.20140101_040000_TAI.1.continuum.fits', 'c:\\Users\\srugi\\Documents\\sdo-clv-pipeline\\data\\hmi.ic_720s.20140101_080000_TAI.1.continuum.fits', 'c:\\Users\\srugi\\Documents\\sdo-clv-pipeline\\data\\hmi.ic_720s.20140101_120000_TAI.1.continuum.fits', 'c:\\Users\\srugi\\Documents\\sdo-clv-pipeline\\data\\hmi.ic_720s.20140101_160000_TAI.1.continuum.fits', 'c:\\Users\\srugi\\Documents\\sdo-clv-pipeline\\data\\hmi.ic_720s.20140101_200000_TAI.1.continuum.fits', 'c:\\Users\\srugi\\Documents\\sdo-clv-pipeline\\data\\hmi.ic_720s.20140102_000000_TAI.1.continuum.fits', 'c:\\Users\\srugi\\Documents\\sdo-clv-pipeline\\data\\hmi.ic_720s.20140102_040000_TAI.1.continuum.fits', 'c:\\Users\\srugi\\Documents\\sdo-clv-pipeline\\data\\hmi.ic_720s.20140102_080000_TAI.1.continuum.fits', 'c:\\Us

In [16]:
for i in range (0, len(con_files)+1):
    # define the paths where files live
    con_file = os.path.join(data_dir, con_files[i])
    mag_file = os.path.join(data_dir, mag_files[i])
    dop_file = os.path.join(data_dir, dop_files[i])
    aia_file = os.path.join(data_dir, aia_files[i])

    # create the SDOImage object instances
    con = SDOImage(con_file)     # continuum, Relative HMI continuum intensity
    mag = SDOImage(mag_file)     # magnetogram, magnetic field strength
    dop = SDOImage(dop_file)     # Dopplergram, LOS velocity
    aia = SDOImage(aia_file)     # AIA 1700 continuum filtergram, Relative 1700 Angstrom Continuum Intensity

    # do the coordinate transforms, etc.
    dop.calc_geometry()
    con.inherit_geometry(dop)
    mag.inherit_geometry(dop)

    aia.rescale_to_hmi(con) # reporject onto plate scale of HMI images

    # calculate the limb darkening correction
    con.calc_limb_darkening()
    aia.calc_limb_darkening()

    # correct magnetogram for foreshortening
    mag.correct_magnetogram()

    # calculate differential rot., meridional circ., obs. vel, grav. redshift, cbs
    dop.correct_dopplergram(fit_cbs=False)

    # set values to nan for mu less than mu_thresh
    mu_thresh = 0.1
    con.mask_low_mu(mu_thresh)
    dop.mask_low_mu(mu_thresh)
    mag.mask_low_mu(mu_thresh)
    aia.mask_low_mu(mu_thresh)

    mask = SunMask(con, mag, dop, aia)
    mask.mask_low_mu(mu_thresh)

    label_moats_on_sun(mask)

KeyboardInterrupt: 